# Information retrieval

## Home Work 2

## Posting List
### Group 1
#### Ali Mojahed - 9812762554
#### Mehrnoosh Navidimehr - 9822762119
#### Minoo Mohaghegh - 9812762270
#### Helia Ghahraman - 9822762437



# Load Dataset

In [1]:
!git clone https://github.com/Text-Mining/Useful-Corpora-for-Text-Mining-in-Persian-Language.git
!unrar x '/content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part01.rar'

fatal: destination path 'Useful-Corpora-for-Text-Mining-in-Persian-Language' already exists and is not an empty directory.

UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part01.rar


Would you like to replace the existing file farsnews.json
1314018076 bytes, modified on 2019-07-16 10:46
with a new one
1314018076 bytes, modified on 2019-07-16 10:46

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 
User break

User break


In [2]:
import json
news = []
counter = 1
for line in open('/content/farsnews.json', 'r', encoding='utf-8-sig'):
  #we only save news body to avoid fullfilling ram
  temp = dict()
  doc = json.loads(line)
  if('NewsBody' not in doc.keys()):
    continue
  temp['NewsBody'] = doc['NewsBody']
  temp['id'] = counter
  counter += 1
  news.append(temp) 


# Pre-processing Pipeline

In [3]:
import pandas as pd
# will be used to remove stopwords from tokens
stopwords = pd.read_fwf('https://raw.githubusercontent.com/sobhe/hazm/master/hazm/data/stopwords.dat', header=None)[0].to_list()

In [4]:
import re
from string import punctuation as punctuation_str

def remove_symbols_and_numbers(content):
  end_of_msg = ('انتهای پیام', 'انتهای‌پیام', '\r\nانتهای\r\nپیام/ک')
  for w in end_of_msg:
      if w in content[-200:]:
          i = content[-200:].find(w)
          content = content[:-(200-i)-1]

  # Remove punctuation & Numbers
 
  content = re.sub(f'[{punctuation_str}؟!،,?،٪×÷»«><]', '', content)
  content = re.sub(f'[0123456789۰١۱۲۳۴۵۶۷۸۹؛–_‘]', '', content)
  
  return content

In [20]:
from IPython.display import clear_output 

def pipeline(text, normalizer, tokenizer, stemmer, method_name, id):
  result = dict()
  
  no_symbol_text = remove_symbols_and_numbers(text)
  normalized_text = normalizer(no_symbol_text)

  tokens = tokenizer(normalized_text)
  # tokens = filter(lambda t: t not in stopwords, tokens)
  tokens = filter(lambda t: len(t) >= 3, tokens)
  tokens = list(tokens)

  stems = [stemmer(word) for word in tokens]

  # removed to avoid ram occupation

  # result['original'] = text
  # result['no_symbol'] = no_symbol_text
  # result['normalized'] = normalized_text
  # result['tokens'] = tokens
  result['stems'] = stems
  # result['method'] = method_name
  result['id'] = id
  # clear_output()
  # print(id)
  return result

# Install Hazm and Preprocessing

In [6]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from hazm import Normalizer, WordTokenizer, Stemmer
hzam_normalizer = Normalizer()
hazm_tokenizer = WordTokenizer()
hazm_stemmer = Stemmer()

In [8]:
results = [pipeline(news[i]['NewsBody'], hzam_normalizer.normalize, hazm_tokenizer.tokenize, hazm_stemmer.stem, 'hazm', i) for i in range(0, 100)]

99


# Posting List





In [9]:

def posting_list(pre_processed_text):
  posting_list = dict()
  for i in range(0, len(pre_processed_text)):
    token_freq_in_doc = dict()
    for token in pre_processed_text[i]['stems']:
      if token not in token_freq_in_doc.keys():
        token_freq_in_doc[token] = 0
      token_freq_in_doc[token] += 1
      if token in posting_list.keys():
        posting_list[token]['freq'] += 1
      else:
        posting_list[token] = dict()
        posting_list[token]['freq'] = 1
    
    for token, freq in token_freq_in_doc.items():
      if 'doc-tf' not in posting_list[token].keys():
        posting_list[token]['doc-tf'] = list()
      
      posting_list[token]['doc-tf'].append((i, freq))
  return posting_list

In [10]:
pl = posting_list(results)

for token, info in pl.items():
  print(f"{token} -> freq: {info['freq']}, posting list: {info['doc-tf']}")

Streaming output truncated to the last 5000 lines.
خواروبار -> freq: 1, posting list: [(2, 1)]
سوپرمارک -> freq: 3, posting list: [(2, 3)]
جه -> freq: 32, posting list: [(2, 1), (4, 3), (12, 1), (13, 1), (15, 2), (21, 2), (32, 3), (38, 2), (39, 1), (40, 2), (44, 2), (72, 1), (88, 1), (90, 4), (92, 2), (93, 2), (97, 1), (99, 1)]
تامین -> freq: 61, posting list: [(2, 3), (13, 2), (24, 5), (33, 6), (38, 32), (39, 1), (56, 1), (71, 2), (72, 1), (76, 1), (88, 1), (93, 5), (99, 1)]
کالا -> freq: 17, posting list: [(2, 7), (13, 2), (38, 3), (42, 1), (49, 1), (56, 1), (93, 1), (95, 1)]
اساس -> freq: 30, posting list: [(2, 4), (4, 2), (18, 1), (20, 1), (21, 2), (23, 1), (25, 1), (27, 1), (33, 1), (34, 1), (48, 1), (56, 1), (59, 1), (71, 1), (72, 1), (79, 1), (83, 5), (90, 2), (97, 2)]
مرد -> freq: 68, posting list: [(2, 2), (5, 1), (10, 1), (17, 3), (21, 3), (23, 1), (24, 8), (25, 2), (29, 1), (32, 1), (33, 5), (34, 2), (38, 2), (45, 2), (46, 3), (50, 7), (53, 1), (59, 3), (63, 1), (65, 4), (72

In [11]:
len(pl.keys())

5266

In [12]:
with open('/content/output.txt', 'w') as file:
  for token, info in pl.items():
    file.write(f"{token} -> freq: {info['freq']}, posting list: {info['doc-tf']}\n")

# Test on All Data

## 1. Run blindly on All Data

this will cause to crash the runtime only on preprocessing

In [16]:
# colab out of memory ~ 200000 docs pre processed

# results_all = [pipeline(news[i]['NewsBody'], hzam_normalizer.normalize, hazm_tokenizer.tokenize, hazm_stemmer.stem, 'hazm', i) for i in range(0, len(news))]
# pl_all = posting_list(results_all)

# with open('/content/output.txt', 'w') as file:
#   for token, info in pl_all.items():
#     file.write(f"{token} -> freq: {info['freq']}, posting list: {info['doc-tf']}\n")

196633


TypeError: ignored

## 2. Parallel Computing

this method will crash the environment too.

In [13]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        ndx = int(ndx)
        yield iterable[ndx:min(ndx + n, l)]

In [14]:
from multiprocessing import cpu_count, Pool

def parallel_process(data, task):
  number_of_threads = cpu_count() - 1
  batches = batch(data, len(data) // number_of_threads)
  tasks_data = []
  print(batches)
  for _batch in batches:
    tasks_data.append(_batch)

  
  pool = Pool(number_of_threads)
  print(pool.map(task, tasks_data))

  


In [15]:

def parallel_pre_process(news):
  result = []
  for i in range(0, len(news)):
    if news[i] is not None:
      result.append(pipeline(news[i]['NewsBody'], hzam_normalizer.normalize, hazm_tokenizer.tokenize, hazm_stemmer.stem, 'hazm', news[i]['id']) for i in range(0, len(news)))
    print(news[i])
  return result

In [ ]:
# parallel_process(news, parallel_pre_process) # colab out of memory

<generator object batch at 0x7f19a2ea0350>


Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.9/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.9/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.9/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: ignored

## 3. run posting list after stemming the document

In [17]:
def posting_list_v2(pre_processed_text, posting_list):
  
  token_freq_in_doc = dict()
  for token in pre_processed_text['stems']:
    if token not in token_freq_in_doc.keys():
      token_freq_in_doc[token] = 0
    token_freq_in_doc[token] += 1
    if token in posting_list.keys():
      posting_list[token]['freq'] += 1
    else:
      posting_list[token] = dict()
      posting_list[token]['freq'] = 1
    
    for token, freq in token_freq_in_doc.items():
      if 'doc-tf' not in posting_list[token].keys():
        posting_list[token]['doc-tf'] = list()
      
      posting_list[token]['doc-tf'].append((id, freq))
  return posting_list

In [22]:
def posting_list_in_place(id, text, posting_list_):
    result = pipeline(text, hzam_normalizer.normalize, hazm_tokenizer.tokenize, hazm_stemmer.stem, 'hazm', id)
    posting_list_ = posting_list_v2(result, posting_list_)
    if id % 20 == 0:
        clear_output()

    if id % 1000 == 0:
        with open('output.txt', 'w') as file:
            for token, info in posting_list_.items():
                file.write(f"{token} -> freq: {info['freq']}, posting list: {info['doc-tf']}\n")
    print(f"processed docs {id}")
    print(f"founded tokens {len(posting_list_.keys())}")
    return posting_list_

In [ ]:
posting_list_ = dict()

for index, data in enumerate(news):
    posting_list_ = posting_list_in_place(index, data['NewsBody'], posting_list_)

posting_list_

processed docs 1880
founded tokens 22295
processed docs 1881
founded tokens 22296
processed docs 1882
founded tokens 22307
processed docs 1883
founded tokens 22310
processed docs 1884
founded tokens 22313
processed docs 1885
founded tokens 22318
processed docs 1886
founded tokens 22321
processed docs 1887
founded tokens 22321
